In [2]:
#Preparation of Dataset
#Taking data from drive
import cv2,os

data_path='/home/suzi/Desktop/Data Science_python/Project_Files/Face Mask Detection/Face Mask Detection-20200623T131306Z-001/Face Mask Detection/dataset'
categories=os.listdir(data_path)

In [3]:
# Labeling category
labels = [i for i in range(len(categories))]
labels_dict = dict(zip(categories,labels))

In [5]:
imag_size = 80
Data_X =[]
Data_Y = []
for category in categories:
    path = os.path.join(data_path,category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        gray=cv2.cvtColor(img_array,cv2.COLOR_BGR2GRAY)
        new_array = cv2.resize(gray,(imag_size,imag_size))
        Data_X.append(new_array)
        Data_Y.append(labels_dict[category])   

In [6]:
import numpy as np
data = np.array(Data_X)
data = np.reshape(data,(len(Data_X),imag_size,imag_size,1))/255.0
Y = np.array(Data_Y)
#Y = np.reshape(Y,(Y.shape[0],1))

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

# model.add(Conv2D(200,(3,3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# # #adding 3rd layer to CNN

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,Y,test_size=0.1)

In [26]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [27]:
history=model.fit(train_data,train_target,epochs=10,callbacks=[checkpoint],validation_data =(test_data, test_target))

Train on 1238 samples, validate on 138 samples
Epoch 1/10
1238/1238 [==============================] - 55s 44ms/sample - loss: 0.6755 - accuracy: 0.5945 - val_loss: 0.5785 - val_accuracy: 0.6739
Epoch 2/10
1238/1238 [==============================] - 50s 40ms/sample - loss: 0.4379 - accuracy: 0.7964 - val_loss: 0.3146 - val_accuracy: 0.8623
Epoch 3/10
1238/1238 [==============================] - 47s 38ms/sample - loss: 0.2332 - accuracy: 0.9095 - val_loss: 0.2117 - val_accuracy: 0.9203
Epoch 4/10
1238/1238 [==============================] - 48s 39ms/sample - loss: 0.1626 - accuracy: 0.9443 - val_loss: 0.1877 - val_accuracy: 0.9203
Epoch 5/10
1238/1238 [==============================] - 48s 38ms/sample - loss: 0.1294 - accuracy: 0.9612 - val_loss: 0.1905 - val_accuracy: 0.9203
Epoch 6/10
1238/1238 [==============================] - 48s 39ms/sample - loss: 0.0920 - accuracy: 0.9685 - val_loss: 0.1229 - val_accuracy: 0.9638
Epoch 7/10
1238/1238 [==============================] - 47s 38ms/

In [50]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

In [56]:
model = load_model('model-009.model')

face_clsfr=cv2.CascadeClassifier('/home/suzi/Desktop/Data Science_python/Project_Files/Face Mask Detection/Face Mask Detection-20200623T131306Z-001/Face Mask Detection/haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'NO MASK',1:'MASK'}
color_dict={0:(0,0,255),1:(0,255,0)}

In [57]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,2,5)  

    for x,y,w,h in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(imag_size,imag_size))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,imag_size,imag_size,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()